In [1]:
import pandas as pd
from pathlib import Path
import win32com.client as win32


In [2]:
EXCEL_FILE_PATH = Path.cwd() / "Financial_Data.xlsx"
ATTACHMENT_DIR = Path.cwd() / "Attachments"

ATTACHMENT_DIR.mkdir(exist_ok=True)

In [3]:
data = pd.read_excel(EXCEL_FILE_PATH,sheet_name="Data")

In [4]:
data.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2021-01-01,1,January,2021
1,Government,Germany,Carretera,None,1321.0,3,20,26420.0,0.0,26420.0,13210.0,13210.0,2021-01-01,1,January,2021
2,Midmarket,France,Carretera,None,2178.0,3,15,32670.0,0.0,32670.0,21780.0,10890.0,2021-06-01,6,June,2021
3,Midmarket,Germany,Carretera,None,888.0,3,15,13320.0,0.0,13320.0,8880.0,4440.0,2021-06-01,6,June,2021
4,Midmarket,Japan,Carretera,None,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021


In [5]:
column_name = "Country"
unique_values = data[column_name].unique()
unique_values

array(['Canada', 'Germany', 'France', 'Japan', 'United States of America'],
      dtype=object)

In [6]:
data.query("Country=='Japan'").head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
4,Midmarket,Japan,Carretera,None,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021
10,Midmarket,Japan,Montana,None,2470.0,5,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021
12,Small Business,Japan,Montana,None,958.0,5,300,287400.0,0.0,287400.0,239500.0,47900.0,2021-08-01,8,August,2021
17,Midmarket,Japan,Paseo,None,974.0,10,15,14610.0,0.0,14610.0,9740.0,4870.0,2021-02-01,2,February,2021
21,Government,Japan,Paseo,None,883.0,10,7,6181.0,0.0,6181.0,4415.0,1766.0,2021-08-01,8,August,2021


In [7]:
for unique_value in unique_values:
    data_output = data.query(f"{column_name}==@unique_value & Year==2021")
    output_path = ATTACHMENT_DIR / f"{unique_value}_2021.xlsx"
    data_output.to_excel(output_path,sheet_name=unique_value,index=False)

In [10]:
email_list = pd.read_excel(EXCEL_FILE_PATH,sheet_name="Email_List")
email_list

,Country,Name,Email,CC
0,Canada,Emma,emma@codingisfun.com;peter@codingisfun.com,sven@codingisfun.com
1,France,Leo,leo@codingisfun.com,sven@codingisfun.com
2,Germany,Lisa,lisa@codingisfun.com,sven@codingisfun.com
3,Japan,Tanaka-san,tanaka@codingisfun.com,sven@codingisfun.com
4,United States of America,Noah,noah@codingisfun.com,sven@codingisfun.com


In [13]:
outlook = win32.Dispatch("outlook.application")
for index, row in email_list.iterrows():
    mail = outlook.CreateItem(0)
    mail.To = row["Email"]
    mail.CC = row["CC"]
    mail.Subject = f"Financial Report for: {row['Country']}"
    mail.HTMLBody = f"""
                    <b>Hi {row['Name']}</b>,<br><br>
                    Please find attached the report for {row['Country']}.<br><br>
                    Best regards,<br>
                    Tommy
                    """
    attachment_path = str(ATTACHMENT_DIR / f"{row['Country']}_2021.xlsx")
    mail.Attachments.Add(Source=attachment_path)
    mail.Display()